# Generative Adversarial Networks (GAN) & Deep Convolutional GAN (DCGAN)
![GAN](https://skymind.ai/images/wiki/gan_schema.png)<br>
The main idea of GAN is easy to understand. We are going to train two networks: one is called the **generator** and the other is called the **discriminator**. The generator takes random inputs ($z$) and creates synthetic data (e.g. images) $$\hat{x}=G(z)$$ which will be passed to the discriminator. The goal of the discriminator is to distinguish the synthetic images from the real ones $$D(x)\in [0,1].$$ We hope that the generator can produce good fake data such that the discriminator fail to find them out $D(G(z))\approx1$.<br> The cost function of GAN is: $$\min_{\theta^{G}}\max_{\theta^{D}}\dfrac{1}{2}E_{x\sim{p_{data}}}\log{D(x)} + \dfrac{1}{2}E_z\log{(1-D(G(z)))}$$ where $\theta^{G}$ are the parameters belong to the generator and $\theta^{D}$ are the parameters belong to the discriminator.
The generator and the discriminator are trained alternatively (train one and keep another fixed!). We hope that the generator and the discriminator can improve their performances simultaneously and smoothly. If one is too good, the other will be difficult to improve itself. [Reference1](https://seas.ucla.edu/~kao/nndl/lectures/gans.pdf)[Reference2](https://skymind.ai/wiki/generative-adversarial-network-gan).

In [1]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Reshape, Input, Flatten,Activation,BatchNormalization
from keras.layers import UpSampling2D,Conv2D, MaxPooling2D,Flatten, Conv2DTranspose
from keras.optimizers import Adam
from keras.datasets import fashion_mnist
import gc
import numpy as np
from PIL import Image
import argparse
import math

Using TensorFlow backend.


In [2]:
# we only requires the training data
(x_train,_),(_,_) = fashion_mnist.load_data()
x_train.shape

(60000, 28, 28)

## Build the generator

In [3]:
def generator(shape=(100,)):
    x_input = Input(shape=shape)
    x = Dense(7*7*128,activation='relu')(x_input)
    x = BatchNormalization()(x)
    x = Reshape((7,7,128))(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2DTranspose(64,(5,5),padding='same',activation='relu')(x)
    x = UpSampling2D((2,2))(x)
    x_output = Conv2DTranspose(1,(5,5),padding='same',activation='tanh')(x)
    model = Model(x_input,x_output)
    return model

## Build the discriminator

In [4]:
def discriminator(shape=(28,28,1)):
    x_input = Input(shape=shape)
    x = Conv2D(64,(5,5),padding='valid')(x_input)
    x = MaxPooling2D((2,2))(x)
    x = Activation(activation='relu')(x)
    x = Conv2D(128,(5,5),padding='valid')(x)
    x = MaxPooling2D((2,2))(x)
    x = Activation(activation='relu')(x)
    x = Flatten()(x)
    x = Dense(1024,activation='relu')(x)
    x_output = Dense(1,activation='sigmoid')(x)
    model = Model(x_input,x_output) 
    return model

## Show the structures of the generator and the discriminator

In [5]:
g_model = generator(shape=(100,))
d_model = discriminator(shape=(28,28,1))
g_model.summary()
d_model.summary()
del g_model, d_model
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
batch_normalization_1 (Batch (None, 6272)              25088     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        204864    
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 64)        0         
__________

108

## Feed the output of generator to the discriminator 

In [6]:
def generator_to_discriminator(g,d):
    d.trainable = False
    x = d(g.output)
    pip_model = Model(g.input,x)
    return pip_model

## Show the fake images
The following codes are modified based on [jacobgil](https://github.com/jacobgil/keras-dcgan).

In [7]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [8]:
def train(BATCH_SIZE):
    # load the data and preprocessing
    (X_train, _), (_, _) = fashion_mnist.load_data()
    X_train = (X_train.reshape(-1,28,28,1).astype(np.float32) - 127.5)/127.5
    
    # create model instances
    d = discriminator()
    g = generator()
    g_to_d = generator_to_discriminator(g, d)
    
    # set the learning rates of d and g
    d_optim = Adam(lr=0.0005)
    g_optim = Adam(lr=0.0005)
    g.compile(loss='binary_crossentropy', optimizer=g_optim)
    g_to_d.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(100):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            # set random inputs
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            # save some generated images
            if index % 500 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    './gimages/' + str(epoch)+"_"+str(index)+".png")
                
            #print(image_batch.shape,generated_images.shape)
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = g_to_d.train_on_batch(noise, [1] * BATCH_SIZE)
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)

In [9]:
train(BATCH_SIZE=42)

Epoch is 0
Number of batches 1428
batch 0 d_loss : 0.704922
batch 0 g_loss : 0.319592
batch 1 d_loss : 0.824473
batch 1 g_loss : 0.477084
batch 2 d_loss : 0.715579
batch 2 g_loss : 0.634255
batch 3 d_loss : 0.660765
batch 3 g_loss : 0.759724
batch 4 d_loss : 0.641801
batch 4 g_loss : 0.882891
batch 5 d_loss : 0.612935
batch 5 g_loss : 1.064131
batch 6 d_loss : 0.585715
batch 6 g_loss : 1.319104
batch 7 d_loss : 0.539036
batch 7 g_loss : 1.413970
batch 8 d_loss : 0.466134
batch 8 g_loss : 1.385486
batch 9 d_loss : 0.473537
batch 9 g_loss : 1.292480
batch 10 d_loss : 0.513270
batch 10 g_loss : 1.151400
batch 11 d_loss : 0.575933
batch 11 g_loss : 0.897568
batch 12 d_loss : 0.551559
batch 12 g_loss : 0.985065
batch 13 d_loss : 0.600349
batch 13 g_loss : 1.218302
batch 14 d_loss : 0.480464
batch 14 g_loss : 1.827442
batch 15 d_loss : 0.405054
batch 15 g_loss : 2.547604
batch 16 d_loss : 0.590380
batch 16 g_loss : 2.406584
batch 17 d_loss : 0.318053
batch 17 g_loss : 2.094728
batch 18 d_los

batch 160 d_loss : 0.535942
batch 160 g_loss : 1.919408
batch 161 d_loss : 0.481058
batch 161 g_loss : 1.607810
batch 162 d_loss : 0.560404
batch 162 g_loss : 1.465890
batch 163 d_loss : 0.565179
batch 163 g_loss : 1.413978
batch 164 d_loss : 0.474290
batch 164 g_loss : 1.453644
batch 165 d_loss : 0.411321
batch 165 g_loss : 1.550653
batch 166 d_loss : 0.281997
batch 166 g_loss : 1.806708
batch 167 d_loss : 0.302118
batch 167 g_loss : 2.133542
batch 168 d_loss : 0.264970
batch 168 g_loss : 2.189323
batch 169 d_loss : 0.304692
batch 169 g_loss : 2.217106
batch 170 d_loss : 0.311653
batch 170 g_loss : 2.191743
batch 171 d_loss : 0.273558
batch 171 g_loss : 2.201013
batch 172 d_loss : 0.287866
batch 172 g_loss : 2.036423
batch 173 d_loss : 0.237670
batch 173 g_loss : 1.903093
batch 174 d_loss : 0.301569
batch 174 g_loss : 1.833599
batch 175 d_loss : 0.361140
batch 175 g_loss : 1.845828
batch 176 d_loss : 0.386579
batch 176 g_loss : 2.073999
batch 177 d_loss : 0.347198
batch 177 g_loss : 2

batch 310 d_loss : 0.478581
batch 310 g_loss : 2.706245
batch 311 d_loss : 0.190839
batch 311 g_loss : 2.463494
batch 312 d_loss : 0.421625
batch 312 g_loss : 1.706143
batch 313 d_loss : 0.423178
batch 313 g_loss : 1.791891
batch 314 d_loss : 0.315555
batch 314 g_loss : 2.620884
batch 315 d_loss : 0.289054
batch 315 g_loss : 3.009590
batch 316 d_loss : 0.202585
batch 316 g_loss : 4.236648
batch 317 d_loss : 0.274099
batch 317 g_loss : 3.854906
batch 318 d_loss : 0.206178
batch 318 g_loss : 3.392314
batch 319 d_loss : 0.376263
batch 319 g_loss : 3.668276
batch 320 d_loss : 0.293517
batch 320 g_loss : 4.475852
batch 321 d_loss : 0.413585
batch 321 g_loss : 4.874224
batch 322 d_loss : 0.368782
batch 322 g_loss : 3.358382
batch 323 d_loss : 0.445965
batch 323 g_loss : 2.595167
batch 324 d_loss : 0.397662
batch 324 g_loss : 2.731104
batch 325 d_loss : 0.230366
batch 325 g_loss : 3.445984
batch 326 d_loss : 0.197094
batch 326 g_loss : 3.658575
batch 327 d_loss : 0.205915
batch 327 g_loss : 3

batch 458 d_loss : 0.194430
batch 458 g_loss : 2.794549
batch 459 d_loss : 0.188876
batch 459 g_loss : 2.794209
batch 460 d_loss : 0.161404
batch 460 g_loss : 2.636843
batch 461 d_loss : 0.128272
batch 461 g_loss : 2.684215
batch 462 d_loss : 0.074953
batch 462 g_loss : 2.554121
batch 463 d_loss : 0.081028
batch 463 g_loss : 2.993161
batch 464 d_loss : 0.077781
batch 464 g_loss : 3.059030
batch 465 d_loss : 0.078135
batch 465 g_loss : 3.700279
batch 466 d_loss : 0.119320
batch 466 g_loss : 4.212538
batch 467 d_loss : 0.094690
batch 467 g_loss : 4.275591
batch 468 d_loss : 0.071462
batch 468 g_loss : 4.116000
batch 469 d_loss : 0.131699
batch 469 g_loss : 3.893863
batch 470 d_loss : 0.149218
batch 470 g_loss : 3.225328
batch 471 d_loss : 0.165433
batch 471 g_loss : 2.963537
batch 472 d_loss : 0.138927
batch 472 g_loss : 2.544993
batch 473 d_loss : 0.254363
batch 473 g_loss : 3.004820
batch 474 d_loss : 0.290467
batch 474 g_loss : 3.679630
batch 475 d_loss : 0.226665
batch 475 g_loss : 4

batch 612 g_loss : 3.117492
batch 613 d_loss : 0.234368
batch 613 g_loss : 4.323592
batch 614 d_loss : 0.162796
batch 614 g_loss : 5.102262
batch 615 d_loss : 0.230860
batch 615 g_loss : 4.201708
batch 616 d_loss : 0.234929
batch 616 g_loss : 3.862305
batch 617 d_loss : 0.277674
batch 617 g_loss : 3.721207
batch 618 d_loss : 0.239819
batch 618 g_loss : 3.891522
batch 619 d_loss : 0.252521
batch 619 g_loss : 3.245576
batch 620 d_loss : 0.175114
batch 620 g_loss : 3.005660
batch 621 d_loss : 0.167240
batch 621 g_loss : 2.997573
batch 622 d_loss : 0.188628
batch 622 g_loss : 3.599541
batch 623 d_loss : 0.278190
batch 623 g_loss : 3.349864
batch 624 d_loss : 0.315727
batch 624 g_loss : 2.894946
batch 625 d_loss : 0.212881
batch 625 g_loss : 2.427479
batch 626 d_loss : 0.385121
batch 626 g_loss : 3.492200
batch 627 d_loss : 0.156933
batch 627 g_loss : 4.091296
batch 628 d_loss : 0.280486
batch 628 g_loss : 3.355774
batch 629 d_loss : 0.178129
batch 629 g_loss : 3.358628
batch 630 d_loss : 0

...

batch 391 g_loss : 5.897320
batch 392 d_loss : 0.180763
batch 392 g_loss : 5.301141
batch 393 d_loss : 0.088820
batch 393 g_loss : 5.138815
batch 394 d_loss : 0.143452
batch 394 g_loss : 4.550120
batch 395 d_loss : 0.114268
batch 395 g_loss : 4.421098
batch 396 d_loss : 0.123040
batch 396 g_loss : 4.359625
batch 397 d_loss : 0.135285
batch 397 g_loss : 4.209192
batch 398 d_loss : 0.236317
batch 398 g_loss : 3.916877
batch 399 d_loss : 0.175113
batch 399 g_loss : 4.054800
batch 400 d_loss : 0.112000
batch 400 g_loss : 3.703609
batch 401 d_loss : 0.206556
batch 401 g_loss : 3.707119
batch 402 d_loss : 0.094458
batch 402 g_loss : 3.549230
batch 403 d_loss : 0.168673
batch 403 g_loss : 3.315170
batch 404 d_loss : 0.181652
batch 404 g_loss : 2.937394
batch 405 d_loss : 0.172439
batch 405 g_loss : 3.266476
batch 406 d_loss : 0.173088
batch 406 g_loss : 2.984349
batch 407 d_loss : 0.205422
batch 407 g_loss : 3.132963
batch 408 d_loss : 0.242479
batch 408 g_loss : 3.166187
batch 409 d_loss : 0

batch 542 d_loss : 0.174647
batch 542 g_loss : 3.307012
batch 543 d_loss : 0.276730
batch 543 g_loss : 3.237098
batch 544 d_loss : 0.172568
batch 544 g_loss : 2.948493
batch 545 d_loss : 0.141311
batch 545 g_loss : 2.854656
batch 546 d_loss : 0.151506
batch 546 g_loss : 2.990836
batch 547 d_loss : 0.167789
batch 547 g_loss : 2.883242
batch 548 d_loss : 0.140953
batch 548 g_loss : 3.448957
batch 549 d_loss : 0.140361
batch 549 g_loss : 3.607247
batch 550 d_loss : 0.132946
batch 550 g_loss : 3.981745
batch 551 d_loss : 0.123056
batch 551 g_loss : 4.291124
batch 552 d_loss : 0.063830
batch 552 g_loss : 4.074430
batch 553 d_loss : 0.100695
batch 553 g_loss : 3.986440
batch 554 d_loss : 0.131235
batch 554 g_loss : 3.974901
batch 555 d_loss : 0.194603
batch 555 g_loss : 3.903103
batch 556 d_loss : 0.105263
batch 556 g_loss : 3.611884
batch 557 d_loss : 0.147857
batch 557 g_loss : 3.381382
batch 558 d_loss : 0.146153
batch 558 g_loss : 3.265296
batch 559 d_loss : 0.180864
batch 559 g_loss : 3

batch 697 d_loss : 0.165406
batch 697 g_loss : 3.049173
batch 698 d_loss : 0.113539
batch 698 g_loss : 2.927119
batch 699 d_loss : 0.149062
batch 699 g_loss : 3.067425
batch 700 d_loss : 0.143338
batch 700 g_loss : 2.696008
batch 701 d_loss : 0.129589
batch 701 g_loss : 3.054437
batch 702 d_loss : 0.179690
batch 702 g_loss : 3.083292
batch 703 d_loss : 0.263030
batch 703 g_loss : 2.731433
batch 704 d_loss : 0.197438
batch 704 g_loss : 2.569095
batch 705 d_loss : 0.155177
batch 705 g_loss : 2.118689
batch 706 d_loss : 0.206457
batch 706 g_loss : 2.209590
batch 707 d_loss : 0.237573
batch 707 g_loss : 2.671844
batch 708 d_loss : 0.205578
batch 708 g_loss : 2.606689
batch 709 d_loss : 0.202135
batch 709 g_loss : 3.062252
batch 710 d_loss : 0.228626
batch 710 g_loss : 3.176731
batch 711 d_loss : 0.218179
batch 711 g_loss : 2.664816
batch 712 d_loss : 0.244006
batch 712 g_loss : 2.828724
batch 713 d_loss : 0.220610
batch 713 g_loss : 2.752925
batch 714 d_loss : 0.299121
batch 714 g_loss : 2

batch 843 g_loss : 3.002607
batch 844 d_loss : 0.200324
batch 844 g_loss : 2.694522
batch 845 d_loss : 0.218613
batch 845 g_loss : 3.516513
batch 846 d_loss : 0.213273
batch 846 g_loss : 3.263923
batch 847 d_loss : 0.183296
batch 847 g_loss : 3.469219
batch 848 d_loss : 0.354192
batch 848 g_loss : 3.851474
batch 849 d_loss : 0.288294
batch 849 g_loss : 3.866122
batch 850 d_loss : 0.481934
batch 850 g_loss : 3.162871
batch 851 d_loss : 0.275830
batch 851 g_loss : 2.626535
batch 852 d_loss : 0.115522
batch 852 g_loss : 2.451717
batch 853 d_loss : 0.189451
batch 853 g_loss : 2.033589
batch 854 d_loss : 0.213274
batch 854 g_loss : 2.112091
batch 855 d_loss : 0.200094
batch 855 g_loss : 2.021811
batch 856 d_loss : 0.233170
batch 856 g_loss : 2.077696
batch 857 d_loss : 0.218114
batch 857 g_loss : 2.407904
batch 858 d_loss : 0.182937
batch 858 g_loss : 2.626363
batch 859 d_loss : 0.187799
batch 859 g_loss : 2.840314
batch 860 d_loss : 0.188939
batch 860 g_loss : 3.602644
batch 861 d_loss : 0

KeyboardInterrupt: 